In [1]:
import torch

from model import Knn_Kmeans_Logits 
from utils.other import load_data
from configs.config import config

In [2]:
device =  torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
model = Knn_Kmeans_Logits(config, device=device)

for i in range(6):

    X_train, y_train, X_test, y_test, covariances = load_data(task_num=i, dataset_name="resnet", load_covariances=True, load_prototypes=False)

    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    acc = (torch.sum((y_test.flatten().to(device)==predictions).int()) / X_test.shape[0] * 100).item()

    print(f"Accuracy: {acc} MY\n")


Accuracy: 83.56, Task ID: 0
WARNING! All values in centroids smaller than 0 were set to 0
Epoch [1|50] Loss: 3.950129288073165 (in this regularization: 0.0) ||| Validation Loss: 3.9416263435460346, Val accuracy: 0.0
Epoch [2|50] Loss: 3.9352648022075813 (in this regularization: 0.0) ||| Validation Loss: 3.9296448864514315, Val accuracy: 0.0
Epoch [3|50] Loss: 3.9253345526064547 (in this regularization: 0.0) ||| Validation Loss: 3.9214249411715736, Val accuracy: 0.0
Epoch [4|50] Loss: 3.918217237003314 (in this regularization: 0.0) ||| Validation Loss: 3.9151866586902475, Val accuracy: 0.0
Epoch [5|50] Loss: 3.7921774067436926 (in this regularization: 0.0) ||| Validation Loss: 2.8769860418536997, Val accuracy: 0.9998000264167786
Epoch [6|50] Loss: 1.3170726592548359 (in this regularization: 0.0) ||| Validation Loss: 0.4553260806995102, Val accuracy: 0.9998000264167786
Epoch [7|50] Loss: 0.25835590531079533 (in this regularization: 0.0) ||| Validation Loss: 0.15340100992706757, Val accur